# Section 0 Defining modules/libraries/functions

In [1]:
import glob
import pandas as pd
import numpy as np
from pathlib import Path
pd.options.display.max_rows = 4000


In [2]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns


# Section 1 Importing Data

In [3]:
cropsCodes = pd.read_csv("C:/Users/cianw/Documents/dataAnalytics/CA2/Data/Eurostat/Code Dictionary/crops.dic", sep='\t',names=['crops', 'crop_name'], header = None)
strucproCodes = pd.read_csv("C:/Users/cianw/Documents/dataAnalytics/CA2/Data/Eurostat/Code Dictionary/strucpro.dic",sep='\t',names=['code', 'units'], header = None)
unitCodes = pd.read_csv("C:/Users/cianw/Documents/dataAnalytics/CA2/Data/Eurostat/Code Dictionary/unit.dic",sep='\t',names=['code', 'units'], header = None)

In [53]:

#Organic Production Data
stdProduction_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\apro_cpsh1_linear.csv")
orgTonne_path= Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\org_croppro_linear.csv")
orgArea_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\org_cropar_linear.csv")

#Harmonised Risk Index 1 Data
hriPath = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Pesiticide Use Risk Indicator\aei_hri_linear.csv")

#Organic Processors Data
orgProcessors_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Organic Processors\org_cpreact_linear.csv")

#Organic Area Utilisation Data
orgAreaUtil_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Organic Area\sdg_02_40_linear.csv")

#N and P Fertilizer Data
fertUse_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Fertlizer Use\aei_fm_usefert_linear.csv")

#Waste Generation Data
wasteGeneration_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Resource Usage\cei_pc034_linear.csv")

#National Productivity Data
productivityIndex_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Resource Usage\cei_pc030_linear.csv")

#Country Gini Data
countryGini_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Quality of Life\tessi190_linear.csv")

#Employment Rate Data
employmentRate_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Economics\tesem010_linear.csv")

#Median/Mean Income Data
income_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Economics\ilc_di03_linear.csv")

#Biodiversity of Birds Data
birdBiodiversity_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Biodiversity Index\env_bio2_linear.csv")

#Pesticide Sales
#pestSales_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Pesticide Sales\aei_fm_salpest09_linear.csv")

#Pesticide Use
pestUse_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\aei_pestuse_linear.csv")

In [5]:
#Harmonised Risk Index 1 Data
hriPesticide = pd.read_csv(hriPath)
hriPesticide = hriPesticide[hriPesticide['subst_cat'].isin(['HRI1'])]
hriPesticide = hriPesticide[~hriPesticide['geo'].isin(['EU', 'EU27_2020', 'EU28'])]
hriPesticide = hriPesticide.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])
hriPesticide = hriPesticide.rename(columns={"OBS_VALUE":"harmRiskInd"})
#print(hriPesticide.groupby('TIME_PERIOD').geo.nunique())
#hriPesticide.shape
#hriPesticide.info()

In [6]:
#Organic Processors
orgProcessors = pd.read_csv(orgProcessors_path)
orgProcessors  = orgProcessors[orgProcessors['nace_r2'].isin(['C103'])]
orgProcessors  = orgProcessors[orgProcessors['unit'].isin(['NR'])]
orgProcessors = orgProcessors.drop(columns = ['DATAFLOW', 'LAST UPDATE','freq', 'unit', 'OBS_FLAG'])
orgProcessors = orgProcessors.rename(columns={"OBS_VALUE":"numOrganicProcessors"})
#print(orgProcessors.groupby('TIME_PERIOD').geo.size())
#print(orgProcessors.groupby('TIME_PERIOD').geo.nunique())
#print(orgProcessors.groupby(['TIME_PERIOD', 'geo']).geo.size())
#orgProcessors.shape
#orgProcessors.info()
#orgProcessors.head()

In [7]:
#Organic Area Utilisation Data
orgAreaUtil = pd.read_csv(orgAreaUtil_path)
orgAreaUtil = orgAreaUtil.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'crops','freq', 'agprdmet', 'unit', 'OBS_FLAG'])
orgAreaUtil = orgAreaUtil.rename(columns={"OBS_VALUE":"areaUsedForOrganic"})
#orgAreaUtil.shape
#orgAreaUtil.info()
#orgAreaUtil.head(10)

Can apply an interesting step here by using left joins to exclude unwanted data later.

The dataset imported and pivoted/transposed below contains more than just the Countries required, it contains subdivisions indicated by numbers beside the country name. Instead of manually writing code that excludes these cases, this set will be left joined later, which will exclude the data automatically.

In [8]:
#N and P Fertilizer Data
fertUse = pd.read_csv(fertUse_path)
fertUse = fertUse.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])
fertUse = fertUse.pivot(index=['geo', 'TIME_PERIOD'], columns='nutrient', values='OBS_VALUE').reset_index() 
fertUse = fertUse.rename(columns={"N":"N_use_tonne",
                                 "P":"P_use_tonne"})
#print(fertUseGroup3.groupby('TIME_PERIOD').geo.nunique())
#fertUse.shape
#fertUse.info()

Below I am imputing the missing odd years by averaging the two around it. By treating the numbers as a trend.

In [9]:
#Waste Generation Data
wasteGeneration = pd.read_csv(wasteGeneration_path)
wasteGeneration = wasteGeneration.drop(columns = ['DATAFLOW', 'LAST UPDATE','nace_r2', 'unit','freq', 'OBS_FLAG', 'hazard', 'waste'])
wasteGeneration["waste_unit"] = "KG per Capita"
wasteGeneration = wasteGeneration.rename(columns={"OBS_VALUE":"waste"})

wasteGeneration_lag = wasteGeneration
wasteGeneration_lag['TIME_LAG'] = wasteGeneration_lag.groupby(['geo'])['TIME_PERIOD'].shift(1)
wasteGeneration_lag['waste_lag'] = wasteGeneration_lag.groupby(['geo'])['waste'].shift(1)

wasteGeneration_lag['waste_temp'] = ((wasteGeneration_lag['waste'] + wasteGeneration_lag['waste_lag'])/2)
wasteGeneration_lag['TIME_PERIOD_temp'] = ((wasteGeneration_lag['TIME_PERIOD'] + wasteGeneration_lag['TIME_LAG'])/2)
wasteGeneration_lag = wasteGeneration_lag.drop(columns = ['TIME_PERIOD', 'TIME_LAG','waste', 'waste_lag'])
wasteGeneration_lag = wasteGeneration_lag.dropna()
wasteGeneration_lag = wasteGeneration_lag.rename(columns={"waste_temp":"waste",
                                                           "TIME_PERIOD_temp":"TIME_PERIOD"})
wasteGeneration_lag['TIME_PERIOD'] = wasteGeneration_lag['TIME_PERIOD'].astype('int')
wasteGeneration = pd.concat([wasteGeneration, wasteGeneration_lag], join='inner', ignore_index=True)

#wasteGeneration.shape
#wasteGeneration.info()

In [10]:
#National Productivity Data
productivityIndex = pd.read_csv(productivityIndex_Path)
productivityIndex  = productivityIndex[productivityIndex['unit'].isin(['PPS_KG'])]
productivityIndex = productivityIndex.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'unit','freq', 'OBS_FLAG'])
productivityIndex["productivity_unit"] = "Purchase Power Standard Per KG"
productivityIndex = productivityIndex.rename(columns={"OBS_VALUE":"productivity"})
#productivityIndex.shape
#productivityIndex.info()
#productivityIndex.head(20)

In [11]:
#GINI index data
countryGini = pd.read_csv(countryGini_path)

countryGini = countryGini.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_il', 'OBS_FLAG'])
countryGini = countryGini.rename(columns={"OBS_VALUE":"gini"})

#countryGini.shape
#countryGini.info()

In [12]:
#Employment Rate Data
emplyomentRate = pd.read_csv(employmentRate_Path)
emplyomentRate = emplyomentRate.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'age', 'unit', 'indic_em'])
emplyomentRate = emplyomentRate.pivot(index=['geo', 'TIME_PERIOD'], columns='sex', values='OBS_VALUE').reset_index() 
emplyomentRate = emplyomentRate.rename(columns={"T":"emplyomentRate_T",
                                               "M":"emplyomentRate_M",
                                               "F":"emplyomentRate_F"})
#emplyomentRate.shape
#emplyomentRate.info()

In [31]:
#Median/Mean Income Data
income = pd.read_csv(income_Path)
income  = income[income['unit'].isin(['EUR'])]
income  = income[income['age'].isin(['TOTAL'])]
income = income.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'age', 'unit'])
income = income.pivot(index=['geo', 'TIME_PERIOD'], columns=['indic_il', 'sex'], values='OBS_VALUE').reset_index() 
income.columns = [''.join(col) for col in income.columns.values]
#income.head()
#income.shape
#income.info()


,geo,TIME_PERIOD,MED_EF,MEI_EF,MED_EM,MEI_EM,MED_ET,MEI_ET
0,AL,2017,1747,2122,1771,2163,1760,2143
1,AL,2018,1972,2310,2015,2367,1997,2339
2,AL,2019,2195,2574,2258,2664,2231,2619
3,AL,2020,2482,2909,2551,2978,2523,2943
4,AT,1995,13394,15196,14654,16256,13971,15708


In [14]:
#Bird Biodiversity
birdBiodiversity = pd.read_csv(birdBiodiversity_Path)
birdBiodiversity = birdBiodiversity.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'unit'])
birdBiodiversity = birdBiodiversity.rename(columns={"OBS_VALUE":"birdBiodiversityIndex"})

#birdBiodiversity.head()

In [77]:
# Pest Sales for Stats Analysis
pestUse= pd.read_csv(pestUse_Path)
#pestUse = pestUse.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG'])
#pestSales = pestSales[(pestSales['TIME_PERIOD'].isin([2016,2018]))]

pestUse['TIME_PERIOD'] = pestUse['TIME_PERIOD'].apply(lambda x: str(x)+ '_y')
pestUse = pestUse[(pestUse['unit'].isin(['KG']))]
pestUse = pestUse[(pestUse['pesticid'].str.contains("_"))]
pestUse  = pestUse[~pestUse['OBS_FLAG'].isin(['c'])]
pestUse['OBS_VALUE'] = np.where(pestUse.OBS_FLAG == 'n', 0, pestUse.OBS_VALUE)
#pestSales = pestSales[(pestSales['pesticid'].str.isdigit()==True)]

#print(pestSales.pesticid.unique())
#pestSales.head()

pestUse = pestUse.pivot(index=['pesticid','crops'], columns =['TIME_PERIOD', 'geo'], values='OBS_VALUE').reset_index() 
#pestUse.columns = ['_'.join(col) for col in pestUse.columns.values]

#pestUse.head(30)

#missing_values_table(pestUse)


TIME_PERIOD,pesticid,crops,2013_y,2014_y,2012_y,2017_y,2015_y,2017_y,2010_y,2017_y,...,2011_y,2015_y,2012_y,2014_y,2015_y,2010_y,2012_y,2017_y,2016_y,2010_y
geo,,,RO,SI,IE,SE,PL,SI,SE,BE,...,IE,IE,IT,IT,IT,IT,CY,CY,FR,FR
0,F01_01,C1110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F01_01,C1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F01_01,C1112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F01_01,C1120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F01_01,C1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,F01_01,C1320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,F01_01,C1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,F01_01,C1600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,F01_01,C2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#Standard Crop Production Import
stdProduction_lin = pd.read_csv(stdProduction_path)
stdProduction_lin = stdProduction_lin[stdProduction_lin['crops'].str.contains("0000")]
stdProduction_lin = stdProduction_lin[stdProduction_lin['strucpro'].isin(['AR', 'PR_HU_EU']) ]
stdProduction_lin  = stdProduction_lin[~stdProduction_lin['OBS_FLAG'].isin(['c','n'])]
stdProduction_lin_yield = stdProduction_lin[stdProduction_lin['strucpro'].isin(['YI_HU_EU']) ]
stdProduction = stdProduction_lin.pivot(index=['crops', 'geo', 'TIME_PERIOD'], columns='strucpro', values='OBS_VALUE').reset_index() 
stdProduction = stdProduction.dropna(subset=['AR', 'PR_HU_EU']) #Removes last NAN value in SET
#stdProduction = stdProduction[(stdProduction['AR'] != 0)] #Removes last NAN value in SET
stdProduction['area_HA'] = stdProduction['AR']*1000
stdProduction['tonnes'] = stdProduction['PR_HU_EU']*1000

In [17]:
#missing_values_table(stdProduction)
#print('Unique Geo:' + str(stdProduction.geo.nunique()))
#print(stdProduction.groupby('TIME_PERIOD').geo.nunique())
#stdProduction.describe()
#stdProduction.info()

In [18]:
del stdProduction_lin 
del stdProduction_lin_yield

In [19]:
#Organic Crop Production Import
orgArea_all = pd.read_csv(orgArea_path)
orgArea_total = orgArea_all[(orgArea_all['agprdmet'] == 'TOTAL') & (orgArea_all['unit']=='HA' ) & orgArea_all['crops'].str.contains("0000")]
orgArea_total = orgArea_total.rename(columns={"OBS_VALUE":"area_HA"})
#orgArea_total.info()
orgArea_total = orgArea_total[~orgArea_total['OBS_FLAG'].isin(['c','n'])]
orgArea_total = orgArea_total.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'agprdmet', 'OBS_FLAG'])
orgArea_total = orgArea_total.dropna(subset=['area_HA']) #Removes last NAN value in SET
orgArea_total = orgArea_total[(orgArea_total['area_HA'] != 0)] #Removes 0 area values in SET which cause inf
#missing_values_table(orgArea_total)
#orgArea_total.head(25)

#del orgArea_all

orgTonne = pd.read_csv(orgTonne_path)
orgTonne = orgTonne[orgTonne['crops'].str.contains("0000")]
orgTonne = orgTonne.rename(columns={"OBS_VALUE":"tonnes"})
#orgTonne.info()
orgTonne = orgTonne[~orgTonne['OBS_FLAG'].isin(['c','n'])]
orgTonne = orgTonne.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])
#missing_values_table(orgTonne)

orgProduction = pd.merge(orgArea_total, orgTonne, on=['crops', 'geo', 'TIME_PERIOD'], how='inner', suffixes=('_A','_T'))
#orgProduction.head(100)

In [20]:
cropProd = pd.merge( stdProduction, orgProduction, on=['crops', 'geo', 'TIME_PERIOD'], how='inner', suffixes=('_std','_org'))
cropProd = pd.merge( cropProd, cropsCodes, on=['crops'], how='inner')
cropProd['geo'] = cropProd['geo'].astype('str') 
#print(cropProd.groupby('TIME_PERIOD').geo.nunique())
#hriPesticide.shape
#cropProd.describe()
#cropProd.info()
#cropProd.head()

In [21]:
#print(orgProduction.geo.unique())
#print(stdProduction.geo.unique())
#print(orgArea_all.geo.unique())
#print(orgTonne.geo.unique())

# Variable Creation

In [22]:
cropProd['tonne_per_HA_org'] = cropProd['tonnes_org']/cropProd['area_HA_org']
cropProd['tonne_per_HA_std'] = cropProd['tonnes_std']/cropProd['area_HA_std']
cropProd['util_ratio'] = cropProd['tonne_per_HA_org']/cropProd['tonne_per_HA_std']

cropProdTotals = cropProd[cropProd['crops'].str.contains("0000")]

cropProdTotals_Geo_Y= cropProdTotals.groupby(['geo', 'TIME_PERIOD']).sum(numeric_only = True).reset_index()
#cropProdTotals_Geo= cropProdTotals.groupby(['geo']).sum(numeric_only = True).reset_index()
#cropProdTotals_crop_Y= cropProdTotals.groupby(['crops', 'crop_name', 'TIME_PERIOD']).sum(numeric_only = True).reset_index()
#cropProdTotals_crop= cropProdTotals.groupby(['crops', 'crop_name']).sum(numeric_only = True).reset_index()

#cropProdTotals_Geo_Y.shape
#cropProdTotals_Geo_Y.head(10)
#print(cropProdTotals.groupby(['TIME_PERIOD', 'geo']).size())

In [23]:
#%whos DataFrame
#hriPesticide.head()
#hriPesticide.shape

In [24]:
import functools as ft
extraVars = [hriPesticide, orgProcessors, orgAreaUtil, countryGini, cropProdTotals_Geo_Y, birdBiodiversity, emplyomentRate, income, fertUse, productivityIndex, wasteGeneration]
rds = ft.reduce(lambda left, right: pd.merge(left,right, how='left', on=['geo', 'TIME_PERIOD']), extraVars)
#rds.shape
#rds.head(20)
#print(extraVars_df.groupby(['TIME_PERIOD', 'geo']).size())
missing_values_table(rds)#Will likely keep NAN values and use as a category when clustered to create scorecard perhaps?

Your selected dataframe has 33 columns.
There are 27 columns that have missing values.


,Missing Values,% of Total Values
tonne_per_HA_std,82,29.4
util_ratio,82,29.4
AR,82,29.4
PR_HU_EU,82,29.4
area_HA_std,82,29.4
tonnes_std,82,29.4
area_HA_org,82,29.4
tonnes_org,82,29.4
tonne_per_HA_org,82,29.4
nace_r2,71,25.4


In [25]:
rds.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\referenceDataSet.csv")

In [26]:
cropProdTotals_Geo_Y.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\cropProdTotals_Geo_Y.csv")
cropProdTotals_crop_Y.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\cropProdTotals_Crop_Y.csv")
pestSales.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\pestSales.csv")

NameError: name 'cropProdTotals_crop_Y' is not defined